In [1]:
import json
import pandas as pd
import numpy as np
import time
import requests

In [2]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [3]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [4]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [0]
    return get_information(method, params)['result'] 

In [5]:
def getStakeAndRewards():
    stake = dict()
    reward = dict()
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['currently-in-committee'] == True:
            address = i['validator']['address']
            reward_accumulated = i['lifetime']['reward-accumulated']
            effective_stake = int(float(i['metrics']['total-effective-stake']))
            stake[address] = effective_stake
            reward[address] = reward_accumulated
    return stake, reward

In [6]:
def getRewards():
    reward = dict()
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['currently-in-committee'] == True:
            address = i['validator']['address']
            reward_accumulated = i['lifetime']['reward-accumulated']
            reward[address] = reward_accumulated
    return reward


In [7]:
def diffAndFilter(map1, map2):
    map3 = dict()
    for k, v in map2.items():
        if k in map1:
            if v - map1[k] != 0:
                map3[k] = v - map1[k]
    return map3

In [8]:
def R11_test():
    print("Test-R11: Earning is proportional to effective stake ")
    num = int(input("enter the number of blocks you want to test: "))
    
    iterations = 0
    while iterations < num:
        print("test " + str(iterations+1) + " will begin ...")
        block = getBlockNumber()
        print("current block number", block, "will begin testing")
        rewards = getRewards()

        new_block = block + 1
        while block < new_block:
            block = getBlockNumber()
        print("new block reached", block, "will calculate the reward per block")
        next_stakes, next_rewards = getStakeAndRewards()
        reward_per_block = diffAndFilter(rewards, next_rewards)

        new_block = block + 1
        while block < new_block:
            block = getBlockNumber()
        print("new block reached", block, "will calculate the difference of reward per block and difference of effective stake")
        new_stakes, new_rewards = getStakeAndRewards()
        reward_per_block_next = diffAndFilter(next_rewards, new_rewards)
        reward_diff = diffAndFilter(reward_per_block, reward_per_block_next)

        # get the validators whose effective stake changes
        stake_diff = diffAndFilter(next_stakes, new_stakes)
        if not stake_diff:
            print("in this iteration, no validators change the effective stake")
    
        for k,v in stake_diff.items():
            if k in reward_diff:
                if v > 0: 
                    if reward_diff[k] <= 0:
                        print("Test-R11: Fail")
                        print("effective stake increase: ", stake_diff)
                        print("reward per block doesn't increase: ", reward_diff)
                    else:
                        print("Test-R11: Success")
                if v < 0:
                    if reward_diff[k] >= 0:
                        print("Test-R11: Fail")
                        print("effective stake decrease: ", stake_diff)
                        print("reward per block doesn't decrease: ", reward_diff)
                    else:
                        print("Test-R11: Success")
        
        new_block = getBlockNumber()
        while new_block == block:
            new_block = getBlockNumber()
        
        iterations += 1        

In [9]:
R11_test()

Test-R11: Earning is proportional to effective stake 
enter the number of blocks you want to test: 3
test 1 will begin ...
current block number 5521 will begin testing
new block reached 5522 will calculate the reward per block
new block reached 5523 will calculate the difference of reward per block and difference of effective stake
in this iteration, no validators change the effective stake
test 2 will begin ...
current block number 5524 will begin testing
new block reached 5525 will calculate the reward per block
new block reached 5526 will calculate the difference of reward per block and difference of effective stake
in this iteration, no validators change the effective stake
test 3 will begin ...
current block number 5527 will begin testing
new block reached 5528 will calculate the reward per block
new block reached 5529 will calculate the difference of reward per block and difference of effective stake
in this iteration, no validators change the effective stake
